In [23]:
import pandas as pd
import numpy as np
import talib
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder

daycandle_dir = 'daycandle'
daycandle_files = os.listdir(daycandle_dir)


#데이터 전처리: 각종 지표 추가, 가격/거래량 -> 변동률로 변환, 코인 심볼을 인코딩 (종류가 많아서 one-hot X)
df_list = []
for file_name in daycandle_files:
    file_path = daycandle_dir + '/' + file_name
    df = pd.read_csv(file_path)
    
    df['candle_date_time_utc'] = pd.to_datetime(df['candle_date_time_utc'], errors='coerce')
    df.sort_values(by='candle_date_time_utc', ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    df['price_change_rate'] = df['trade_price'].pct_change()
    df['volume_change_rate'] = df['candle_acc_trade_volume'].pct_change()
    
    df['coin_symbol'] = df['market'].str.replace('KRW-', '')
    close = df['trade_price'].values
    high = df['high_price'].values
    low = df['low_price'].values
    open_ = df['opening_price'].values
    volume = df['candle_acc_trade_volume'].values

    df['EMA_12'] = talib.EMA(close, timeperiod=12)
    df['SMA_60'] = talib.SMA(close, timeperiod=60)
    df['ADX'] = talib.ADX(high, low, close, timeperiod=14)
    df['OBV'] = talib.OBV(close, volume)

    macd, macdsignal, macdhist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'] = macd
    df['MACDSIGNAL'] = macdsignal
    df['MACDHIST'] = macdhist

    upperband, middleband, lowerband = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    df['UpperBand'] = upperband
    df['MiddleBand'] = middleband
    df['LowerBand'] = lowerband
    
    df.dropna(subset=['SMA_60', 'EMA_12', 'MACD', 'ADX', 'UpperBand', 'OBV'], inplace=True)    
    df_list.append(df)


# final_df = pd.concat(df_list, ignore_index=True)
processed_df = pd.concat(df_list).reset_index(drop=True)
processed_df.set_index('candle_date_time_utc', inplace=True)

le = LabelEncoder()
processed_df['coin_id'] = le.fit_transform(processed_df['coin_symbol'])

processed_df.sort_values(by=['coin_symbol', processed_df.index.name], inplace=True)
processed_df.head()

,market,opening_price,high_price,low_price,trade_price,candle_acc_trade_price,candle_acc_trade_volume,change_rate,price_change_rate,volume_change_rate,...,SMA_60,ADX,OBV,MACD,MACDSIGNAL,MACDHIST,UpperBand,MiddleBand,LowerBand,coin_id
candle_date_time_utc,,,,,,,,,,,,,,,,,,,,,
2021-12-25,KRW-1INCH,3370.0,3440.0,3290.0,3315.0,2.257582e+10,6.739325e+06,-0.014859,-0.014859,-0.617149,...,4413.833333,33.501043,1.379928e+08,-266.794583,-367.952399,101.157816,3450.717625,3179.50,2908.282375,0
2021-12-26,KRW-1INCH,3320.0,3380.0,3205.0,3340.0,1.568185e+10,4.754130e+06,0.007541,0.007541,-0.294569,...,4352.500000,31.836117,1.427470e+08,-238.371865,-342.036292,103.664427,3456.801653,3181.50,2906.198347,0
2021-12-27,KRW-1INCH,3340.0,3585.0,3305.0,3405.0,2.406820e+10,6.989608e+06,0.019461,0.019461,0.470218,...,4305.583333,29.628569,1.497366e+08,-208.201680,-315.269370,107.067689,3468.047174,3184.50,2900.952826,0
2021-12-28,KRW-1INCH,3405.0,3430.0,3085.0,3115.0,2.084798e+10,6.374463e+06,-0.085169,-0.085169,-0.088008,...,4256.666667,28.198932,1.433621e+08,-205.325305,-293.280557,87.955251,3439.381253,3170.75,2902.118747,0
2021-12-29,KRW-1INCH,3115.0,3170.0,2910.0,2975.0,1.540664e+10,5.079641e+06,-0.044944,-0.044944,-0.203127,...,4213.250000,27.401732,1.382825e+08,-211.899937,-277.004433,65.104496,3444.520870,3163.00,2881.479130,0


In [40]:
#지표 생성으로 생긴 결측치 row 제거 + sequence 생성을 위한 window_size + 예측원하는 날짜 범위가 총 row의 10% 미만인 경우 필터링
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

WINDOW_SIZE = 60
PREDICTION_HORIZONS = [1, 3, 7]
max_horizon = max(PREDICTION_HORIZONS)
min_sequence_length_per_sample = WINDOW_SIZE + max_horizon


min_total_length_for_valid_splits = min_sequence_length_per_sample / min(TRAIN_RATIO, VAL_RATIO, TEST_RATIO)
MIN_ROW_COUNT = 2700
coin_lengths = processed_df.groupby('coin_symbol').size()
long_enough_coins = coin_lengths[(coin_lengths >= min_total_length_for_valid_splits) & (coin_lengths >= MIN_ROW_COUNT)].index


filtered_df = processed_df[processed_df['coin_symbol'].isin(long_enough_coins)].copy()

print(f"총 row: {len(filtered_df)}")
print(f"제외된 종목 수: {len(coin_lengths) - len(long_enough_coins)}")

총 row: 55988
제외된 종목 수: 144


In [25]:
#데이터 분할

#각 종목별로 훈련 테스트 검증 데이터를 고루 분포시키기 위해
def split_data_by_coin(coin_df_group):
    total_len = len(coin_df_group)
    train_len = int(total_len * TRAIN_RATIO)
    val_len = int(total_len * VAL_RATIO)
    
    train_split = coin_df_group.iloc[:train_len]
    val_split = coin_df_group.iloc[train_len : train_len + val_len]
    test_split = coin_df_group.iloc[train_len + val_len :]
    
    return train_split, val_split, test_split


train_dfs, val_dfs, test_dfs = [], [], []
for symbol, group_df in filtered_df.groupby('coin_symbol'):
    train_part, val_part, test_part = split_data_by_coin(group_df)
    train_dfs.append(train_part)
    val_dfs.append(val_part)
    test_dfs.append(test_part)


train_df = pd.concat(train_dfs)
val_df = pd.concat(val_dfs)
test_df = pd.concat(test_dfs)

In [26]:
#스케일링
feature_cols = [
    'opening_price',
    'high_price',
    'low_price',
    'trade_price',
    'candle_acc_trade_price',
    'candle_acc_trade_volume',
    'price_change_rate',
    'volume_change_rate',
    'EMA_12',
    'SMA_60',
    'ADX',
    'OBV',
    'MACD',
    'MACDSIGNAL',
    'MACDHIST',
    'UpperBand',
    'MiddleBand',
    'LowerBand',
]
target_col = 'price_change_rate'

X_train_features = train_df[feature_cols]
coin_ids_train = train_df['coin_id']

X_val_features = val_df[feature_cols]
coin_ids_val = val_df['coin_id']

X_test_features = test_df[feature_cols]
coin_ids_test = test_df['coin_id']


feature_scaler = StandardScaler()
X_train_scaled = feature_scaler.fit_transform(X_train_features)
X_val_scaled = feature_scaler.transform(X_val_features)
X_test_scaled = feature_scaler.transform(X_test_features)

In [28]:
import numpy as np
import pandas as pd

WINDOW_SIZE = 60
PREDICTION_HORIZONS = [1, 3, 7]

def create_sequences(features_np_array, coin_ids_series, window_size, prediction_horizons):
    X_seq, y_seq, coin_ids_seq = [], [], []
    max_horizon = max(prediction_horizons)
    
    for symbol_id in coin_ids_series.unique():
        features_for_symbol = features_np_array[coin_ids_series.values == symbol_id]

        if len(features_for_symbol) < window_size + max_horizon:
            continue

        for i in range(len(features_for_symbol) - window_size - max_horizon + 1):
            X_seq.append(features_for_symbol[i : i + window_size, :])
            
            y_targets_for_horizons = []
            for h in prediction_horizons:
                y_targets_for_horizons.append(features_for_symbol[i + window_size + h - 1, :])
            y_seq.append(y_targets_for_horizons)
            
            coin_ids_seq.append(symbol_id)

    return np.array(X_seq), np.array(y_seq), np.array(coin_ids_seq)

X_train_seq, y_train_seq, train_coin_ids_seq = create_sequences(
    X_train_scaled, coin_ids_train, WINDOW_SIZE, PREDICTION_HORIZONS
)
X_val_seq, y_val_seq, val_coin_ids_seq = create_sequences(
    X_val_scaled, coin_ids_val, WINDOW_SIZE, PREDICTION_HORIZONS
)
X_test_seq, y_test_seq, test_coin_ids_seq = create_sequences(
    X_test_scaled, coin_ids_test, WINDOW_SIZE, PREDICTION_HORIZONS
)

--- Row counts per coin ID (sorted ascending) ---
coin_id
67     1667
89     1738
61     1765
142    1772
19     1776
4      1782
114    1799
10     1802
87     1805
137    1810
97     1833
36     1850
116    1863
154    1891
138    1900
34     1911
83     1928
53     1933
33     1939
91     1944
62     1950
105    1950
76     1994
77     1996
46     1996
75     2002
24     2004
112    2047
163    2047
161    2063
106    2068
121    2074
63     2084
141    2091
71     2101
1      2102
113    2204
14     2228
3      2233
15     2242
130    2242
124    2242
127    2242
129    2242
157    2242
158    2242
153    2242
117    2242
100    2242
103    2242
55     2242
25     2242
86     2242
65     2242
56     2242
35     2244
Name: count, dtype: int64
-------------------------------------------------
--- Row counts per coin ID (sorted ascending) ---
coin_id
67     208
89     217
61     220
142    221
4      222
19     222
114    224
10     225
87     225
137    226
97     229
36     231
116 

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, X_data, y_data, id_data):
        self.X = torch.tensor(X_data, dtype=torch.float32)
        self.y = torch.tensor(y_data, dtype=torch.float32)
        self.id_seq = torch.tensor(id_data, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return {
            "past_values": self.X[idx],
            "future_values": self.y[idx],
            "static_categorical_features": self.id_seq[idx]
        }

train_dataset = TimeSeriesDataset(
    X_train_seq,
    y_train_seq,
    train_coin_ids_seq
)
val_dataset = TimeSeriesDataset(
    X_val_seq,
    y_val_seq,
    val_coin_ids_seq
)
test_dataset = TimeSeriesDataset(
    X_test_seq,
    y_test_seq,
    test_coin_ids_seq
)

BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [32]:

from transformers import PatchTSTConfig, PatchTSTForPrediction, Trainer, TrainingArguments
import numpy as np

config = PatchTSTConfig(
    num_input_channels=len(feature_cols),
    context_length=WINDOW_SIZE,
    prediction_length=len(PREDICTION_HORIZONS),
    patch_length=10,
    stride=10,
    d_model=128,
    num_attention_heads=4,
    num_hidden_layers=3,
    ffn_dim=256,
    dropout=0.1,
    num_static_categorical_features=filtered_df['coin_id'].nunique(),
    cardinality=[filtered_df['coin_id'].nunique()],
    embedding_dimension=[16],
    loss="mse",
)

model = PatchTSTForPrediction(config)

print(f"\n모델 파라미터 수: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

training_args = TrainingArguments(
    output_dir="./patchtst_results",
    num_train_epochs=50,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=1e-4,
    weight_decay=1e-5,
    eval_strategy="epoch",
    logging_dir="./patchtst_logs",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="tensorboard",
    fp16=True,
    dataloader_num_workers=10
)

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    mae = np.mean(np.abs(preds - labels))
    rmse = np.sqrt(np.mean((preds - labels)**2))
    return {"mae": mae, "rmse": rmse}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


모델 파라미터 수: 399235


In [8]:
trainer.train()

RuntimeError: DataLoader worker (pid(s) 14428, 13676, 3132, 10548, 16292, 16308, 8320, 4752, 10256, 15344) exited unexpectedly